In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import sys
sys.path.append("../")
from constants import *

In [4]:
movies_df = pd.read_csv("../data/movies.csv")

In [5]:
len(movies_df)

9742

In [6]:
movies_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
ratings_df = pd.read_csv("../data/ratings.csv")

In [8]:
len(ratings_df)

100836

In [9]:
ratings_df.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [10]:
ratings_df.sort_values(by=["timestamp"], inplace=True)
grouped_ratings = ratings_df.groupby(by="userId").agg(list)

In [11]:
grouped_ratings.head(10)

,movieId,rating,timestamp
userId,,,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744...","[4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, ...","[964980499, 964980499, 964980523, 964980523, 9..."
2,"[318, 79132, 131724, 115713, 99114, 112552, 35...","[3.0, 4.0, 5.0, 3.5, 3.5, 4.0, 4.0, 3.5, 4.0, ...","[1445714835, 1445714841, 1445714851, 144571485..."
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3...","[3.5, 2.0, 3.0, 0.5, 0.5, 0.5, 0.5, 0.5, 5.0, ...","[1306463323, 1306463559, 1306463561, 130646356..."
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5...","[3.0, 4.0, 4.0, 2.0, 3.0, 2.0, 2.0, 5.0, 2.0, ...","[945078428, 945078446, 945078467, 945078467, 9..."
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ...","[5.0, 3.0, 3.0, 2.0, 5.0, 4.0, 4.0, 3.0, 3.0, ...","[847434747, 847434747, 847434748, 847434748, 8..."
6,"[592, 590, 380, 296, 150, 349, 165, 153, 344, ...","[3.0, 5.0, 4.0, 2.0, 4.0, 5.0, 3.0, 3.0, 3.0, ...","[845553109, 845553109, 845553110, 845553110, 8..."
7,"[1784, 2683, 1584, 1610, 1923, 1208, 1220, 191...","[0.5, 2.0, 5.0, 4.0, 1.5, 4.0, 4.5, 4.0, 3.5, ...","[1106635416, 1106635420, 1106635423, 110663542..."
8,"[296, 380, 592, 590, 150, 153, 593, 231, 318, ...","[4.0, 5.0, 3.0, 5.0, 4.0, 2.0, 4.0, 4.0, 5.0, ...","[839463422, 839463422, 839463422, 839463422, 8..."
9,"[3328, 3173, 41, 223, 1674, 4558, 2023, 1037, ...","[3.0, 2.0, 3.0, 4.0, 4.0, 1.0, 3.0, 2.0, 2.0, ...","[1044656650, 1044656650, 1044656650, 104465665..."


In [12]:
movies_df[movies_df.movieId==804]

,movieId,title,genres
632,804,She's the One (1996),Comedy|Romance


In [13]:
item = grouped_ratings.iloc[0]
item

movieId      [804, 1210, 2628, 2826, 2018, 3578, 3617, 3744...
rating       [4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, ...
timestamp    [964980499, 964980499, 964980523, 964980523, 9...
Name: 1, dtype: object

In [14]:
# item.movieId

In [15]:
# movies_df[movies_df.movieId.isin(item.movieId)]

In [16]:
user_movies = {_:row.movieId for _, row in grouped_ratings.iterrows()}

In [17]:
# user_movies[1]

In [18]:
ratings_user_1 = ratings_df.groupby(by="userId").get_group(1)

In [19]:
context_history = 120
val_context_history = 5
end_index = random.randint(10, ratings_user_1.shape[0] - val_context_history)

In [20]:
end_index

127

In [21]:
start_index = max(0, end_index - context_history)
start_index

7

In [22]:
context = ratings_user_1[start_index:end_index]

In [23]:
context

,userId,movieId,rating,timestamp
227,1,3744,4.0,964980694
6,1,101,5.0,964980868
24,1,441,4.0,964980868
184,1,2858,5.0,964980868
96,1,1473,4.0,964980875
...,...,...,...,...
66,1,1127,4.0,964982513
155,1,2414,3.0,964982513
134,1,2115,5.0,964982529
32,1,590,4.0,964982546


In [24]:
trg = context.movieId.tolist()

In [25]:
masked_list = [a if random.random() < 0.8 else TRAIN_CONSTANTS.MASK for a in trg]

In [26]:
len(masked_list)

120

In [49]:
# padding list "left"
mode = "left"
if len(masked_list) < context_history:
    if mode == "left":
        masked_integers = [TRAIN_CONSTANTS.PAD] * (
            context_history - len(masked_list)) + masked_list
    else:
        masked_integers = masked_list + [TRAIN_CONSTANTS.PAD] * (
            context_history - len(masked_list))


In [54]:
len(movies_df) + 2

9744

In [27]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
ratings_df.head()

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66716,429,588,5.0,828124615
66717,429,590,5.0,828124615
66718,429,592,5.0,828124615
66712,429,432,3.0,828124615


In [32]:
movieIdMapping = {k:i+2 for i, k in enumerate(sorted(list(ratings_df.movieId.unique())))}
inverseMovieIdMapping = {v:k for k, v in movieIdMapping.items()}

In [33]:
ratings_df["movieId_mapped"] = ratings_df.movieId.map(movieIdMapping)

In [34]:
ratings_df.head()

,userId,movieId,rating,timestamp,movieId_mapped
66719,429,595,5.0,828124615,514
66716,429,588,5.0,828124615,508
66717,429,590,5.0,828124615,510
66718,429,592,5.0,828124615,511
66712,429,432,3.0,828124615,378


In [35]:
ratings_df.movieId.max()

193609

In [36]:
ratings_df.movieId.min()

1

In [37]:
ratings_df.to_csv("../data/ratings_mapped.csv", index=False)

In [38]:
movies_df["movieId_mapped"] = movies_df.movieId.map(movieIdMapping)

In [39]:
movies_df.head()

,movieId,title,genres,movieId_mapped
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.0
2,3,Grumpier Old Men (1995),Comedy|Romance,4.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,5.0
4,5,Father of the Bride Part II (1995),Comedy,6.0


In [40]:
len(movies_df)

9742

In [41]:
movies_df.movieId.max()

193609

In [42]:
movies_df.movieId.min()

1

In [43]:
ratings_df.movieId.max()

193609

In [44]:
ratings_df.movieId.min()

1

In [45]:
movies_df.movieId_mapped.isna().sum()

18

In [46]:
movies_df.to_csv("../data/movies_mapped.csv", index=False)


In [49]:
import pickle
with open("../data/new_to_old_movie_id_mapping.pkl", "wb") as fp:
    pickle.dump(inverseMovieIdMapping, fp)

In [50]:
ratings_df.movieId_mapped.max()

9725

In [51]:
ratings_df.movieId_mapped.min()

2